# Collaborative Filtering과 Cosine Distance를 활용한 음악 추천

Last.fm 데이터는 스페인 바로셀로나의 Music Technology Group@Universitat Pompeu Fabra에서 만들었다.

이 데이터는 다음 두 부분으로 나뉘어져 있다.

1) 사용자 활동 데이터<br> 2) 사용자 프로필 데이터<br><br>
360,000명의 사용자가 Last.fm에서 들은 아티스트 정보를 포함한다.

먼저, 필요한 라이브러리를 로드하고 옵션을 지정한다.

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
# display results to 3 decimal points, not in scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

사용자의 활동 정보를 불러온다.<br>사용자,아티스트 이름,플레이 횟수로 구성되어 있다.

In [8]:
user_data = pd.read_table('./Data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv',
                          header = None, nrows = 2e7,
                          names = ['users', 'musicbrainz-artist-id', 'artist-name', 'plays'],
                          usecols = ['users', 'artist-name', 'plays'])

사용자의 프로필을 불러온다.

In [9]:
user_profiles = pd.read_table('./data/lastfm-dataset-360K/usersha1-profile.tsv',
                          header = None,
                          names = ['users', 'gender', 'age', 'country', 'signup'],
                          usecols = ['users', 'country'])

In [10]:
user_data.head()

,users,artist-name,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706


In [11]:
user_profiles.head()

,users,country
0,00000c289a1829a808ac09c00daf10bc3c4e223b,Germany
1,00001411dc427966b17297bf4d69e7e193135d89,Canada
2,00004d2ac9316e22dc007ab2243d6fcb239e707d,Germany
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,Mexico
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,United States


데이터를 메모리로 불러왔고, 데이터 전처리 후에 추천 알고리즘을 적용해보자.

### 유명한 아티스트 데이터 추출

추천은 사용자가 들은 아티스트의 패턴을 기반으로 한다.<br>
여러 번 플레이 되지 않은 덜 유명한 아티스트의 경우, 패턴이 정확하지 않을 수 있다. 이 경우 올바르지 않은 추천을 하게 된다. <br> 이 문제를 줄이기 위해 잘 알려진 아티스트의 데이터만 활용한다.

In [12]:
if user_data['artist-name'].isnull().sum() > 0:
    user_data = user_data.dropna(axis = 0, subset = ['artist-name'])

유명한 아티스트를 찾기 위해 각 아티스트 마다 플레이 횟수를 찾아내야 한다.<br> 사용자의 플레이 횟수는 사용자 활동 데이터 한 줄에 하나의 아티스트의 플레이 횟수를 포함하므로 아티스트 단위로 취합해야 한다.

In [14]:
artist_plays = (user_data.
     groupby(by = ['artist-name'])['plays'].
     sum().
     reset_index().
     rename(columns = {'plays': 'total_artist_plays'})
     [['artist-name', 'total_artist_plays']]
    )
artist_plays.head()

,artist-name,total_artist_plays
0,04)],6
1,2,1606
2,58725ab=>,23
3,80lİ yillarin tÜrkÇe sÖzlÜ aŞk Şarkilari,70
4,amy winehouse,23


총 플레이 회수를 사용자의 활동 데이터에 병합해보자.
이 정보를 활용해서 잘 알려지지 않은 아티스트를 필터링할 수 있다.

In [16]:
user_data_with_artist_plays = user_data.merge(artist_plays, left_on = 'artist-name', right_on = 'artist-name', how = 'left')
user_data_with_artist_plays.head()

,users,artist-name,plays,total_artist_plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137,25651
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099,3704875
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897,180391
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717,410725
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706,90498


### 유명한 아티스트 판단 기준 설정

거의 300,000 아티스트들이 있기 때문에, 대부분의 아티스들은 몇 번 정도의 플레이 회수를 가지고 있다. 통계량을 살펴보자.

In [17]:
print(artist_plays['total_artist_plays'].describe())

count     292364.000
mean       12907.037
std       185981.313
min            1.000
25%           53.000
50%          208.000
75%         1048.000
max     30466827.000
Name: total_artist_plays, dtype: float64


가정했던 대로 중앙값은 200 정도 밖에 되지 않는다. 상위 1% 에 대한 통계를 살펴 보자.

In [18]:
print(artist_plays['total_artist_plays'].quantile(np.arange(.9, 1, .01)))

0.900     6138.000
0.910     7410.000
0.920     9102.960
0.930    11475.590
0.940    14898.440
0.950    19964.250
0.960    28419.880
0.970    43541.330
0.980    79403.440
0.990   198482.590
Name: total_artist_plays, dtype: float64


상위 1% 정도가 거의 200,000번 이상 플레이 되었고, 상위 2%가 80,000번, 상위 3%가 40,000번 정도 플레이되었다. 상위 3퍼센트 정도만 추천에 사용하자. 이 기준 이상으로 추천을 하는 경우 적절하지 않은 결과를 가져올 수 있다.

In [19]:
popularity_threshold = 40000
user_data_popular_artists = user_data_with_artist_plays.query('total_artist_plays >= @popularity_threshold')
user_data_popular_artists.head()

,users,artist-name,plays,total_artist_plays
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099,3704875
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897,180391
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717,410725
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706,90498
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691,13547741


### 한국 사용자의 활동 정보만 사용

활동 정보가 담긴 데이터와 사용자의 국가 정보가 담긴 데이터를 병합한다.<br> 데이터 중에 사용자의 국가 정보가 한국인 데이터만 필터링한다.

In [38]:
combined = user_data_popular_artists.merge(user_profiles, left_on = 'users', right_on = 'users', how = 'left')
kor_data = combined.query('country == \'Korea, Republic of\'')
kor_data.head(30)

,users,artist-name,plays,total_artist_plays,country
1471,00065b397ff3c9ac3ef5e2b58f96c1284265bdc1,machinefabriek,475,59579,"Korea, Republic of"
1472,00065b397ff3c9ac3ef5e2b58f96c1284265bdc1,hood,471,139777,"Korea, Republic of"
1473,00065b397ff3c9ac3ef5e2b58f96c1284265bdc1,tape,471,45444,"Korea, Republic of"
1474,00065b397ff3c9ac3ef5e2b58f96c1284265bdc1,sonic youth,455,3967343,"Korea, Republic of"
1475,00065b397ff3c9ac3ef5e2b58f96c1284265bdc1,the fall,448,1033200,"Korea, Republic of"
1476,00065b397ff3c9ac3ef5e2b58f96c1284265bdc1,john fahey,425,240948,"Korea, Republic of"
1477,00065b397ff3c9ac3ef5e2b58f96c1284265bdc1,maher shalal hash baz,413,74771,"Korea, Republic of"
1478,00065b397ff3c9ac3ef5e2b58f96c1284265bdc1,tim hecker,374,461187,"Korea, Republic of"
1479,00065b397ff3c9ac3ef5e2b58f96c1284265bdc1,stars of the lid,321,664611,"Korea, Republic of"
1480,00065b397ff3c9ac3ef5e2b58f96c1284265bdc1,sun city girls,315,144953,"Korea, Republic of"


중복되는 정보가 있으면 제거한다.

In [39]:
if not kor_data[kor_data.duplicated(['users', 'artist-name'])].empty:
    initial_rows = kor_data.shape[0]

    print('Initial dataframe shape {0}'.format(kor_data.shape))
    kor_data = kor_data.drop_duplicates(['users', 'artist-name'])
    current_rows = kor_data.shape[0]
    print('New dataframe shape {0}'.format(kor_data.shape))
    print('Removed {0} rows'.format(initial_rows - current_rows))

Initial dataframe shape (13802, 5)
New dataframe shape (13801, 5)
Removed 1 rows


### Nearest Neighbor Model 구현

#### 데이터 형태 변환(Reshaping)

In [41]:
wide_artist_data = kor_data.pivot(index = 'artist-name', columns = 'users', values = 'plays').fillna(0)
wide_artist_data_sparse = csr_matrix(wide_artist_data.values)

#### 모델링

모델을 만들 차례다. NearestNeighbors 클래스를 초기화하여 model_knn에 넣고 앞서 작성한 sparse matrix를 피팅한다. Metric은 Cosine을 사용하며, 아티스트 벡터 사이의 유사도를 Cosine similarity를 이용해 구하는 것을 의미한다.

In [42]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(wide_artist_data_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

#### 아티스트 추천 생성

이제 아티스트 추천을 해보자. 랜덤하게 뽑은 한 아티스트와 유사한 아티스트 목록을 출력한다.

In [66]:
query_index = np.random.choice(wide_artist_data_sparse.shape[0])
distances, indices = model_knn.kneighbors(wide_artist_data.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(wide_artist_data.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, wide_artist_data.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for the smashing pumpkins:

1: bajofondo, with distance of 0.3025156182926413:
2: david holmes, with distance of 0.3025156182926413:
3: sweatshop union, with distance of 0.3025156182926413:
4: jean grae, with distance of 0.3025548048876874:
5: green day, with distance of 0.3047404599140564:


플레이 회수가 아닌 전에 플레이를 했는지 여부를 가지고 추천하는 알고리즘을 만들어보자.

In [55]:
wide_artist_data_zero_one = wide_artist_data.apply(np.sign)
wide_artist_data_zero_one_sparse = csr_matrix(wide_artist_data_zero_one.values)

In [56]:
model_nn_binary = NearestNeighbors(metric='cosine', algorithm='brute')
model_nn_binary.fit(wide_artist_data_zero_one_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [68]:
distances, indices = model_nn_binary.kneighbors(wide_artist_data_zero_one.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations with binary play data for {0}:\n'.format(wide_artist_data_zero_one.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, wide_artist_data_zero_one.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations with binary play data for the smashing pumpkins:

1: radiohead, with distance of 0.6110777658687012:
2: pixies, with distance of 0.7083333333333334:
3: beck, with distance of 0.716930741463851:
4: damien rice, with distance of 0.7384428128264067:
5: oasis, with distance of 0.7421446884353017:


smashing pumpkins를 좋아하는 사람은 radiohead나 pixies를 좋아할 수도 있습니다. <br> 이전 실험에 비해 거리는 더 크지만 sign 함수를 사용하여 데이터를 스쿼시하기 때문이다.